In [3]:
import pandas, os, copy, ase
import numpy as np
from webbrowser import open as open_log
from math import pi
from IPython.core.display import HTML, Image
from ase.io import read, write
from gausspy import Gaussian
from glob import glob
from ase_extensions.ase_utils import *
from cc_notebook import *
from gaussian_job_manager import on_server
from oniom_utils import oniom_stable
#from adsy import display as ad_disp
#ad_disp.extended_styles()

/Applications/anaconda/envs/clyde/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


OSError: [Errno 13] Permission denied: '/usr/local/share/jupyter'

In [4]:
import IPython.html.nbextensions as nb
ext= 'https://github.com/ipython-contrib/IPython-notebook-extensions/archive/master.zip'
nb.install_nbextension(ext)

OSError: [Errno 13] Permission denied: '/usr/local/share/jupyter'

In [3]:
serv_oniom_stable = on_server(oniom_stable)

In [4]:
cd oniom_stable

/home/clyde/Dropbox/Project Stuff/Notebooks/test_cases/oniom_stable


In [7]:
azo_44_ts_xyz = '../../graphene_cycloadditions/cycloadd_oniom/oniom_xyz_files/non_oniom_ts/B3LYP_631Gd_B3LYP_631Gd_4_4_cent_azomethine_ts_v2_final.xyz'
carb_44_ts_xyz = '../../graphene_cycloadditions/cycloadd_oniom/oniom_xyz_files/non_oniom_ts/B3LYP_631Gd_B3LYP_631Gd_4_4_cent_carbonyl_ts3_final.xyz'

ylide_atoms = [67,68,69,70,71,72,73,74]
graph_atoms = {}
graph_atoms.update({0:[28,29]}) #2 C atoms
graph_atoms.update({1:graph_atoms[0] + [12,13,14,15,27,41,42,30]}) #10 C atoms
graph_atoms.update({2:graph_atoms[1] + [11,25,26,16,32,31]}) #16 C atoms
graph_atoms.update({3:graph_atoms[2] + [24,33]})#18 C atoms
graph_atoms.update({4:graph_atoms[2] + [38,39,52,40,47,55,48,56,43,44,53,45,17,6,60,5,4,59,3,58,2,1,57,10]}) # 42 C atoms (total molecule = 50 C atoms)

In [8]:
azo_ts_test = read(azo_44_ts_xyz)
azo_ts_test.set_calculator(Gaussian(label='azo_ts_test', method='oniom(B3LYP/6-31G(d):pm6)', basis='oniom'))
azo_ts_test_H = ylide_atoms + graph_atoms[3]
azo_ts_test.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_test_H]]
azo_ts_test.calc.oniom_coord_params['layer_mults'] = [1,1]
azo_ts_test.calc.set_job(nodes=12, memory=12*1600, time=5, queue='pqmb')
#azo_ts_test.calc.start(frc=True)
#check_calcs([azo_ts_test])

Initial attempt to construct a custom route failed so instead we will use OnlyInputFiles to generate the individual calculations and edit those. (But look at the end for a working version of an oniom stable calculation using a custom route)

In [9]:
azo_ts_stable = read(azo_44_ts_xyz)
azo_ts_stable.set_calculator(Gaussian(label='azo_ts_stable_2', method='oniom(B3LYP/6-31G(d):pm6)=OnlyInputFiles', basis='oniom'))
azo_ts_stable_H = ylide_atoms + graph_atoms[3]
azo_ts_stable.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_test_H]]
azo_ts_stable.calc.oniom_coord_params['layer_mults'] = [1,1]
azo_ts_stable.calc.set_job(nodes=1, memory=100, time=1, queue='pqmb')
#azo_ts_stable.calc.start(frc=True)

In [10]:
high_mult_azo_ts_stable = read(azo_44_ts_xyz)
high_mult_azo_ts_stable.set_calculator(Gaussian(label='azo_ts_stable_2_mult_3', method='oniom(UB3LYP/6-31G(d):upm6)=OnlyInputFiles', basis='oniom'))
high_mult_azo_ts_stable_H = ylide_atoms + graph_atoms[3]
high_mult_azo_ts_stable.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_test_H]]
high_mult_azo_ts_stable.calc.oniom_coord_params['layer_mults'] = [3,3]
high_mult_azo_ts_stable.calc.set_job(nodes=1, memory=100, time=1, queue='pqmb')
#high_mult_azo_ts_stable.calc.start(frc=True)

In [8]:
low_mult_azo_ts_stable = read(azo_44_ts_xyz)
low_mult_azo_ts_stable.set_calculator(Gaussian(label='azo_ts_stable_2_mult_1', method='oniom(UB3LYP/6-31G(d):upm6)=OnlyInputFiles', basis='oniom'))
low_mult_azo_ts_stable_H = ylide_atoms + graph_atoms[3]
low_mult_azo_ts_stable.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_test_H]]
low_mult_azo_ts_stable.calc.oniom_coord_params['layer_mults'] = [1,1]
low_mult_azo_ts_stable.calc.set_job(nodes=1, memory=100, time=1, queue='pqmb')
#low_mult_azo_ts_stable.calc.start(frc=True)

In [9]:
check_calcs([azo_ts_stable, high_mult_azo_ts_stable, low_mult_azo_ts_stable])
mols_to_html([azo_ts_stable, high_mult_azo_ts_stable, low_mult_azo_ts_stable])

azo_ts_stable_2,com,,Success
azo_ts_stable_2_mult_3,com,,Success
azo_ts_stable_2_mult_1,com,,Success


In [10]:
current_dir = os.getcwd()
home_dir = os.environ['ASE_HOME']
work_dir = os.environ['ASE_SCRATCH']
work_dir + current_dir.split(home_dir)[1] + '/' + low_mult_azo_ts_stable.calc.label + '.log'

'/home/clyde/Dropbox/Project Stuff/Notebooks/test_cases/oniom_stable/azo_ts_stable_2_mult_1.log'

In [11]:
#inp_str=oniom_utils.extract_oniom_inputs(azo_ts_stable)[0]
#mod_inp_str = oniom_utils.add_to_input(inp_str, stable='opt')
low_real, high_model, low_model = oniom_utils.oniom_comp_calcs(azo_ts_stable, stable='opt')
low_real_mult_3, high_model_mult_3, low_model_mult_3 = oniom_utils.oniom_comp_calcs(high_mult_azo_ts_stable, stable='opt')
low_real_mult_1, high_model_mult_1, low_model_mult_1 = oniom_utils.oniom_comp_calcs(low_mult_azo_ts_stable, stable='opt')
#low_real2, high_model2, low_model2 = oniom_utils.oniom_comp_calcs_v2(azo_ts_stable, stable='opt')

In [12]:
for c in [low_real, high_model, low_model, low_real_mult_3, high_model_mult_3, low_model_mult_3, low_real_mult_1, high_model_mult_1, low_model_mult_1]:
    c.calc.set_job(nodes=8, memory=8*1500, time=8, queue='pqmb')
    #c.calc.start(frc=True)

In [13]:
check_calcs([low_real, high_model, low_model, low_real_mult_3, high_model_mult_3, low_model_mult_3, low_real_mult_1, high_model_mult_1, low_model_mult_1])
mols_to_html([low_real, high_model, low_model, low_real_mult_3, high_model_mult_3, low_model_mult_3, low_real_mult_1, high_model_mult_1, low_model_mult_1], data_func=lambda c:c.calc.max_data['scfenergies'][-1] /ase.units.Hartree)

azo_ts_stable_2_low_real,com,,Success,0.318791016754
azo_ts_stable_2_high_model,com,,Error: No lower point found -- run aborted.,-826.931073275
azo_ts_stable_2_low_model,com,,Success,0.223015929404
azo_ts_stable_2_mult_3_low_real,com,,Error: Convergence failure -- run terminated.,0.392794542899
azo_ts_stable_2_mult_3_high_model,com,,Success,-826.932416724
azo_ts_stable_2_mult_3_low_model,com,,Error: Convergence failure -- run terminated.,0.246527575599
azo_ts_stable_2_mult_1_low_real,com,,Success,0.31879101678
azo_ts_stable_2_mult_1_high_model,com,,Error: No lower point found -- run aborted.,-826.931073275
azo_ts_stable_2_mult_1_low_model,com,,Success,0.223015929402


In [14]:
print(high_model_mult_1.calc._get_input(high_model_mult_1))

%mem=100MB
%chk=/work/cjf05/ASE_notebooks/test_cases/oniom_stable/azo_ts_stable_2_mult_1_high_model.chk
%nproc=1
 #P Test IOp(2/15=1,5/32=2,5/38=1) UB3LYP/6-31G(D)
 Geom=Connect gfprint pop=full  stable=opt

 Gaussian input prepared by ASE
 Point  2 -- high level on model system.

     0     1
  Bq-#1(Iso=12)                                     2.806559714088     -3.675033922252     -0.260594371350
  H(Iso=12)                                         1.228388174711     -3.345053753402     -0.122053393892
  Bq-#1(Iso=12)                                     0.671558372503     -4.918894760532     -0.218566431111
  Bq-#1(Iso=12)                                    -0.683290257571     -4.920907303416     -0.227524859570
  H(Iso=12)                                        -1.244044295836     -3.348074963353     -0.123991896006
  Bq-#1(Iso=12)                                    -2.820536300670     -3.678095923209     -0.246767136001
  Bq-#1(Iso=12)                                     7.064931721

Let's see try a simpler higher model string to see if we still get an inability to find the stable wavefunction:

In [15]:
raw_inp = """#P UB3LYP/6-31G(D) stable=opt

 Gaussian input prepared by ASE
 Point  2 -- high level on model system.

     0     1
  H(Iso=12)                                         1.228388174711     -3.345053753402     -0.122053393892
  H(Iso=12)                                        -1.244044295836     -3.348074963353     -0.123991896006
  H(Iso=12)                                         5.462384793887     -0.863378298974     -0.412471097683
  H(Iso=12)                                         3.346059116695     -2.109114998909     -0.250230123282
  C                                                 2.823220607144     -1.207764609738     -0.163061053446
  C                                                 1.418839230223     -1.212737469139      0.013633943420
  C                                                 0.700553760911     -2.449136854885     -0.022836171237
  C                                                -0.718311039779     -2.445883732607     -0.014753947938
  C                                                -1.444759431640     -1.234277107823      0.082535421028
  C                                                -2.837521383703     -1.219509863545     -0.078213429688
  H(Iso=12)                                        -3.362847824843     -2.118758454135     -0.191414898388
  H(Iso=12)                                        -5.481588589689     -0.880022873253     -0.344112780389
  H(Iso=12)                                         5.462097521228      0.934128083895     -0.366390643971
  C                                                 4.946908883064      0.033821577303     -0.334114520183
  C                                                 3.533832222317      0.029177933241     -0.188777801883
  C                                                 2.821924160333      1.263587297947     -0.095842545545
  C                                                 1.416708646284      1.261352698994      0.071423470361
  C                                                 0.728288548825      0.021363617113      0.232245815452
  C                                                -0.740765214007      0.022504793627      0.495177729891
  C                                                -1.446573752563      1.276885327443      0.080786012505
  C                                                -2.838399731450      1.260589350561     -0.075899720555
  C                                                -3.559800588702      0.020163356885     -0.113418521258
  C                                                -4.961500547561      0.020198302758     -0.274782512855
  H(Iso=12)                                        -5.481447714476      0.920728060521     -0.342183851445
  H(Iso=12)                                         3.344457832687      2.168553942176     -0.147935961619
  H(Iso=12)                                         1.221023683205      3.397166766817     -0.010986445232
  C                                                 0.693522269498      2.498250222749      0.056809757981
  C                                                -0.721334787766      2.493003138933      0.022896066250
  H(Iso=12)                                        -1.247891128719      3.395339313689     -0.075341093535
  H(Iso=12)                                        -3.363689890227      2.160858903310     -0.182293695811
  C                                                -0.938950953558      0.103442619453      2.247284952125
  C                                                 1.308277634328     -0.386029349224      2.818416865041
  N                                                 0.019687019660     -0.718926114469      2.866741086009
  H                                                -0.750973912422      1.149361200007      2.491294585679
  H                                                -1.959674603167     -0.191605440962      2.483228710921
  H                                                 2.056499727489     -1.144409100217      3.002438381959
  H                                                 1.573454006168      0.662028090250      2.833552031300
  H                                                -0.188000304646     -1.713751103613      2.874680568695


"""

simple_h_model = copy.deepcopy(high_model_mult_1)
simple_h_model.set_calculator(Gaussian(label='simple_high_model', raw_input=raw_inp))
simple_h_model.calc.set_job(nodes=8, memory=8*1200, queue='pqmb', version='g09')
simple_h_model.calc.initialize(simple_h_model)
simple_h_model.calc.extra_params['raw_input'] = simple_h_model.calc._get_link0() + simple_h_model.calc.extra_params['raw_input']
#simple_h_model.calc.start()

In [16]:
check_calcs([simple_h_model])
mols_to_html([simple_h_model])

simple_high_model,com,,Error: No lower point found -- run aborted.


In [17]:
raw_inp_v2 = """#P UB3LYP/6-31G(D) stable=opt guess=mix

 Gaussian input prepared by ASE
 Point  2 -- high level on model system.

     0     1
  H(Iso=12)                                         1.228388174711     -3.345053753402     -0.122053393892
  H(Iso=12)                                        -1.244044295836     -3.348074963353     -0.123991896006
  H(Iso=12)                                         5.462384793887     -0.863378298974     -0.412471097683
  H(Iso=12)                                         3.346059116695     -2.109114998909     -0.250230123282
  C                                                 2.823220607144     -1.207764609738     -0.163061053446
  C                                                 1.418839230223     -1.212737469139      0.013633943420
  C                                                 0.700553760911     -2.449136854885     -0.022836171237
  C                                                -0.718311039779     -2.445883732607     -0.014753947938
  C                                                -1.444759431640     -1.234277107823      0.082535421028
  C                                                -2.837521383703     -1.219509863545     -0.078213429688
  H(Iso=12)                                        -3.362847824843     -2.118758454135     -0.191414898388
  H(Iso=12)                                        -5.481588589689     -0.880022873253     -0.344112780389
  H(Iso=12)                                         5.462097521228      0.934128083895     -0.366390643971
  C                                                 4.946908883064      0.033821577303     -0.334114520183
  C                                                 3.533832222317      0.029177933241     -0.188777801883
  C                                                 2.821924160333      1.263587297947     -0.095842545545
  C                                                 1.416708646284      1.261352698994      0.071423470361
  C                                                 0.728288548825      0.021363617113      0.232245815452
  C                                                -0.740765214007      0.022504793627      0.495177729891
  C                                                -1.446573752563      1.276885327443      0.080786012505
  C                                                -2.838399731450      1.260589350561     -0.075899720555
  C                                                -3.559800588702      0.020163356885     -0.113418521258
  C                                                -4.961500547561      0.020198302758     -0.274782512855
  H(Iso=12)                                        -5.481447714476      0.920728060521     -0.342183851445
  H(Iso=12)                                         3.344457832687      2.168553942176     -0.147935961619
  H(Iso=12)                                         1.221023683205      3.397166766817     -0.010986445232
  C                                                 0.693522269498      2.498250222749      0.056809757981
  C                                                -0.721334787766      2.493003138933      0.022896066250
  H(Iso=12)                                        -1.247891128719      3.395339313689     -0.075341093535
  H(Iso=12)                                        -3.363689890227      2.160858903310     -0.182293695811
  C                                                -0.938950953558      0.103442619453      2.247284952125
  C                                                 1.308277634328     -0.386029349224      2.818416865041
  N                                                 0.019687019660     -0.718926114469      2.866741086009
  H                                                -0.750973912422      1.149361200007      2.491294585679
  H                                                -1.959674603167     -0.191605440962      2.483228710921
  H                                                 2.056499727489     -1.144409100217      3.002438381959
  H                                                 1.573454006168      0.662028090250      2.833552031300
  H                                                -0.188000304646     -1.713751103613      2.874680568695


"""

simple_h_model_v2 = copy.deepcopy(high_model_mult_1)
simple_h_model_v2.set_calculator(Gaussian(label='simple_high_model_v2', raw_input=raw_inp_v2))
simple_h_model_v2.calc.set_job(nodes=8, memory=8*1200, queue='pqmb', version='g09')
simple_h_model_v2.calc.initialize(simple_h_model)
simple_h_model_v2.calc.extra_params['raw_input'] = simple_h_model_v2.calc._get_link0() + simple_h_model_v2.calc.extra_params['raw_input']
#simple_h_model_v2.calc.start()

In [18]:
check_calcs([simple_h_model_v2])
mols_to_html([simple_h_model_v2])

simple_high_model_v2,com,,Success


In [19]:
raw_inp_v3 = """#P UB3LYP/6-31G(D) stable=opt guess=mix

 Gaussian input prepared by ASE
 Point  2 -- high level on model system.

     0     3
  H(Iso=12)                                         1.228388174711     -3.345053753402     -0.122053393892
  H(Iso=12)                                        -1.244044295836     -3.348074963353     -0.123991896006
  H(Iso=12)                                         5.462384793887     -0.863378298974     -0.412471097683
  H(Iso=12)                                         3.346059116695     -2.109114998909     -0.250230123282
  C                                                 2.823220607144     -1.207764609738     -0.163061053446
  C                                                 1.418839230223     -1.212737469139      0.013633943420
  C                                                 0.700553760911     -2.449136854885     -0.022836171237
  C                                                -0.718311039779     -2.445883732607     -0.014753947938
  C                                                -1.444759431640     -1.234277107823      0.082535421028
  C                                                -2.837521383703     -1.219509863545     -0.078213429688
  H(Iso=12)                                        -3.362847824843     -2.118758454135     -0.191414898388
  H(Iso=12)                                        -5.481588589689     -0.880022873253     -0.344112780389
  H(Iso=12)                                         5.462097521228      0.934128083895     -0.366390643971
  C                                                 4.946908883064      0.033821577303     -0.334114520183
  C                                                 3.533832222317      0.029177933241     -0.188777801883
  C                                                 2.821924160333      1.263587297947     -0.095842545545
  C                                                 1.416708646284      1.261352698994      0.071423470361
  C                                                 0.728288548825      0.021363617113      0.232245815452
  C                                                -0.740765214007      0.022504793627      0.495177729891
  C                                                -1.446573752563      1.276885327443      0.080786012505
  C                                                -2.838399731450      1.260589350561     -0.075899720555
  C                                                -3.559800588702      0.020163356885     -0.113418521258
  C                                                -4.961500547561      0.020198302758     -0.274782512855
  H(Iso=12)                                        -5.481447714476      0.920728060521     -0.342183851445
  H(Iso=12)                                         3.344457832687      2.168553942176     -0.147935961619
  H(Iso=12)                                         1.221023683205      3.397166766817     -0.010986445232
  C                                                 0.693522269498      2.498250222749      0.056809757981
  C                                                -0.721334787766      2.493003138933      0.022896066250
  H(Iso=12)                                        -1.247891128719      3.395339313689     -0.075341093535
  H(Iso=12)                                        -3.363689890227      2.160858903310     -0.182293695811
  C                                                -0.938950953558      0.103442619453      2.247284952125
  C                                                 1.308277634328     -0.386029349224      2.818416865041
  N                                                 0.019687019660     -0.718926114469      2.866741086009
  H                                                -0.750973912422      1.149361200007      2.491294585679
  H                                                -1.959674603167     -0.191605440962      2.483228710921
  H                                                 2.056499727489     -1.144409100217      3.002438381959
  H                                                 1.573454006168      0.662028090250      2.833552031300
  H                                                -0.188000304646     -1.713751103613      2.874680568695


"""

simple_h_model_v3 = copy.deepcopy(high_model_mult_1)
simple_h_model_v3.set_calculator(Gaussian(label='simple_high_model_v3', raw_input=raw_inp_v3))
simple_h_model_v3.calc.set_job(nodes=8, memory=8*1200, queue='pqmb', version='g09')
simple_h_model_v3.calc.initialize(simple_h_model_v3)
simple_h_model_v3.calc.extra_params['raw_input'] = simple_h_model_v3.calc._get_link0() + simple_h_model_v3.calc.extra_params['raw_input']
#simple_h_model_v3.calc.start()

In [20]:
azo_ts = read(azo_44_ts_xyz)
azo_ts.set_calculator(Gaussian(label='azo_ts', method='oniom(UB3LYP/6-31G(d):upm6)', basis='oniom'))
azo_ts_H = ylide_atoms + graph_atoms[3]
azo_ts.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_H]]
azo_ts.calc.oniom_coord_params['layer_mults'] = [1,1]
azo_ts.calc.set_job(nodes=12, memory=12*1600, time=5, queue='pqmb')
#azo_ts.calc.start()
check_calcs([azo_ts])
mols_to_html([azo_ts])

azo_ts,com,,Success


In [21]:
unstable_oniom_e = azo_ts.calc.energy_zero

In [22]:
azo_ts.calc.restart(method='oniom(UB3LYP/6-31G(d):upm6)', component_calcs = [low_real_mult_1, None, low_model_mult_1])
#azo_ts.calc.start(frc=True)
check_calcs([azo_ts],frc=True)
mols_to_html([azo_ts])

azo_ts_restart_1,com,,Success


In [23]:
stable_oniom_e = azo_ts.calc.energy_zero

In [24]:
azo_ts.calc.extra_params['component_calcs']

[Atoms(symbols='C50H23N', positions=..., cell=[1.0, 1.0, 1.0], pbc=[False, False, False], calculator=Gaussian(...)),
 None,
 Atoms(symbols='C50H23N', positions=..., cell=[1.0, 1.0, 1.0], pbc=[False, False, False], calculator=Gaussian(...))]

from oniom calculation:

In [25]:
ad_disp.html_dict({'low_real':0.318791017286, 'high_model': -826.931073274533, 'low_model':0.223015929096})

high_model,low_real,low_model
-826.931073274533,0.318791017286,0.223015929096


vs. from components calculations:

In [26]:
ad_disp.html_dict({'low_real':low_real_mult_1.calc.max_data['scfenergies'][-1] / ase.units.Hartree, 'high_model':high_model_mult_1.calc.max_data['scfenergies'][-1] / ase.units.Hartree, 'low_model':low_model_mult_1.calc.max_data['scfenergies'][-1] / ase.units.Hartree})

high_model,low_real,low_model
-826.931073275,0.31879101678,0.22301592940200002


Good we are successfully reading in the component chk-point files. Let's also make sure restarting an oniom calculation (using guess=read) reads in all the component wavefunctions:

In [27]:
azo_ts.calc.restart(component_calcs = [])
check_calcs([azo_ts])
mols_to_html([azo_ts])

azo_ts_restart_2,com,,Success


In [28]:
ad_disp.html_dict({'low_real':0.318791017297, 'high_model': -826.931073274, 'low_model':0.223015929097})  

high_model,low_real,low_model
-826.931073274,0.318791017297,0.223015929097


Yes it does read them all in. Let's now try combining all the steps (extracting the individual inputs, running stable=opt calculations for each, then loading the subsequent chk point files back into the original oniom calculation) and make sure the resultant function works.

In [29]:
azo_ts_2 = read(azo_44_ts_xyz)
azo_ts_2.set_calculator(Gaussian(label='azo_ts_combined_stable', method='oniom(UB3LYP/6-31G(d):upm6)', basis='oniom'))
azo_ts_2_H = ylide_atoms + graph_atoms[3]
azo_ts_2.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_2_H]]
azo_ts_2.calc.oniom_coord_params['layer_mults'] = [1,1]
azo_ts_2.calc.set_job(nodes=8, memory=8*1400, time=5, queue='pqmb')

#serv_oniom_stable(azo_ts_2,frc=True)

In [30]:
check_calcs([azo_ts_2], frc=True)
mols_to_html([azo_ts_2])

azo_ts_combined_stable,com,,Success


In [31]:
mols_to_html(azo_ts_2.calc.components)

azo_ts_combined_stable_init_low_real,com,,Success
azo_ts_combined_stable_init_high_model_restart_1,com,,Success
azo_ts_combined_stable_init_low_model,com,,Success


In [32]:
auto_stable_oniom_e = azo_ts_2.calc.energy_zero
restarted_stable_oniom_e = azo_ts.calc.energy_zero

manual_stable_oniom_e = low_real_mult_1.calc.max_data['scfenergies'][-1] + high_model_mult_1.calc.max_data['scfenergies'][-1] - low_model_mult_1.calc.max_data['scfenergies'][-1]
ad_disp.html_dict({'Auto constructed stable Oniom Energy': auto_stable_oniom_e /ase.units.Hartree, 'Unstable Oniom Energy':unstable_oniom_e / ase.units.Hartree, 'Stable Oniom Energy':stable_oniom_e / ase.units.Hartree, 'Restarted Stable Oniom Energy': restarted_stable_oniom_e / ase.units.Hartree, 'Manually calculated Stable Oniom Energy': manual_stable_oniom_e / ase.units.Hartree})

Restarted Stable Oniom Energy,Manually calculated Stable Oniom Energy,Auto constructed stable Oniom Energy,Stable Oniom Energy,Unstable Oniom Energy
-826.8352981862,-826.835298187622,-826.845825863713,-826.835298186342,-826.807758620865


Good, it does. Now can we run stability calculations on oniom calculations that are reading from a chk point file?

In [33]:
azo_ts.calc.restart(method='oniom(UB3LYP/6-31G(d):upm6)=OnlyInputFiles')
azo_ts.calc.set_job(nodes=1, memory=100, time=1, queue='pqmb')
#azo_ts.calc.start()
check_calcs([azo_ts])

'Calculations complete'

In [34]:
low_real_m1_v2, high_model_m1_v2, low_model_m1_v2 = oniom_utils.oniom_comp_calcs(azo_ts, stable='opt')
low_real_m1_v2.calc.extra_params['raw_input'].split('\n\n')[0:2]

['%mem=100MB\n%chk=/work/cjf05/ASE_notebooks/test_cases/oniom_stable/azo_ts_restart_3_low_real.chk\n%nproc=1\n #P Test IOp(2/15=1,5/32=2,5/38=1) UPM6\n Geom=Connect gfprint pop=full  stable=opt',
 ' Gaussian input prepared by ASE\n Point  3 -- low level on real system.']

In [35]:
for m in [low_real_m1_v2, high_model_m1_v2, low_model_m1_v2]:
    m.calc.set_job(nodes=8, memory=8*1500, time=8, queue='pqmb')
    #m.calc.start()
check_calcs([low_real_m1_v2, high_model_m1_v2, low_model_m1_v2])
mols_to_html([low_real_m1_v2, high_model_m1_v2, low_model_m1_v2])

azo_ts_restart_3_low_real,com,,Success
azo_ts_restart_3_high_model,com,,Error: No lower point found -- run aborted.
azo_ts_restart_3_low_model,com,,Success


Re-testing automated stability calculation

In [36]:
azo_ts_2_v2 = copy.deepcopy(azo_ts_2)
azo_ts_2_v2.set_calculator(Gaussian(label='azo_ts_combined_stable_v2', method='oniom(UB3LYP/6-31G(d):upm6)', basis='oniom'))
azo_ts_2_H_v2 = ylide_atoms + graph_atoms[3]
azo_ts_2_v2.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_2_H_v2]]
azo_ts_2_v2.calc.oniom_coord_params['layer_mults'] = [1,1]
azo_ts_2_v2.calc.set_job(nodes=8, memory=8*1400, time=5, queue='pqmb')
#serv_oniom_stable(azo_ts_2_v2, frc=True)

'4259114'

In [37]:
check_calcs([azo_ts_2_v2],frc=True)
mols_to_html([azo_ts_2_v2])

SyntaxError: invalid syntax (<ipython-input-37-a1ed5e25a1ba>, line 1)

In [ ]:
mols_to_html(azo_ts_2_v2.calc.components)

In [ ]:
azo_ts_2_v2.calc.components[1].calc.basis

<font color='red', size=5>No we can't =OnlyInputFiles does not use guess=read to extract the wavefunction for that component from the previous oniom chk file even if we are using guess=read in the oniom calculation</font>

How about we try putting 'read' in the method?

In [ ]:
azo_ts.calc.restart(method='oniom(UB3LYP/6-31G(d)=read:upm6=read)=OnlyInputFiles')
azo_ts.calc.set_job(nodes=1, memory=100, time=1, queue='pqmb')
#azo_ts.calc.start()
check_calcs([azo_ts])
mols_to_html([azo_ts])

This doesn't work

What does opt='opt' in combination with '=OnlyInputFiles' do?

In [ ]:
azo_ts.calc.restart(no_old_chk=True, opt='opt', geom='', guess='', method='oniom(UB3LYP/6-31G(d):upm6)=OnlyInputFiles')
azo_ts.calc.set_job(nodes=1, memory=100, time=1, queue='pqmb')
#azo_ts.calc.start()

In [ ]:
check_calcs([azo_ts])
mols_to_html([azo_ts])

This also fails - does some strange mixture of trying to produce input files and also running an optimisation that ultimately results in an error

In [ ]:
azo_ts_3 = read(azo_44_ts_xyz)
azo_ts_3.set_calculator(Gaussian(label='azo_ts_2', method='oniom(UB3LYP/6-31G(d):upm6)', basis='oniom', guess='(always, mix)'))
azo_ts_3_H = ylide_atoms + graph_atoms[3]
azo_ts_3.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_H]]
azo_ts_3.calc.oniom_coord_params['layer_mults'] = [1,1]
azo_ts_3.calc.set_job(nodes=12, memory=12*1600, time=5, queue='pqmb')
#azo_ts_3.calc.start()
#check_calcs([azo_ts_3])

In [ ]:
#print(azo_ts_stable.calc._get_mol_details(azo_ts_stable))

In [ ]:
inp_str = '\n'.join(inp_str.split('\n')[7:])
with open('test.xyz', 'w') as fl:
    fl.write(xyz_str)
pybel.readfile('xyz', 'test.xyz').next()

In [ ]:
import gaussian_input_parser


In [ ]:
with open('test.com', 'w') as fl:
    fl.write(inp_str)

In [ ]:
gaussian_input_parser.extract_calc('test.com')

In [ ]:
from ase import Atoms, Atom
from ase.calculators.gaussian import Gaussian
import re

In [ ]:
from gaussian_input_parser import extract_calc
atoms=extract_calc('test.com')

In [ ]:
coords[2]

In [ ]:
with open(inp_file, 'r') as fl:
    data = fl.read()

section_names = ['route', 'title', 'coords', 'connects']
sections = data.split('\n\n')

no_sections = len(sections)
data_dict = dict(zip(section_names[:no_sections], sections))

label = data_dict.get('title')
route = data_dict.get('route')
coords = data_dict.get('coords')
#connects = data_dict.get('connects')

if coords:
    charge, mult = coords.split('\n')[0].split()
    coords = '\n'.join(coords.split('\n')[1:])

atoms = Atoms()
if coords:
    for line in coords.split('\n'):
        info = line.split()
        symbol = info[0]
        position = [float(info[1]), float(info[2]), float(info[3])]
        atoms += Atom(symbol, position=position)

    if not label:
        label = 'Default Gaussian Label'

route_args = route.split()[1:]
route_kwargs = {}
for route_arg in route_args:
    if  re.match('IOP', route_arg, re.I):
        route_kwargs.update({'ioplist': route_arg[4:-1].split(',')})
    elif '=' in route_arg:
        route_arg_nm, route_arg_vl = route_arg.split('=')
        route_kwargs.update({route_arg_nm: route_arg_vl})
    else:
        route_kwargs.update({route_arg:route_arg})

atoms.set_calculator(Gaussian(label=label, multiplicity=mult, **route_kwargs))

## Routes:

In [11]:
trial_rt = """1/38=1,52=2/1;
2/12=2,17=6,18=5,40=1/2;
1/38=1,52=2,53=3172/20;
3/5=2,16=1,25=1,41=3900000/1,2,3;
4/35=1/1;
5/5=2,35=1,38=5/2;
8/6=1,10=90,11=11/1;
9/8=-1,42=1,49=10/14;
6/7=2,8=2,9=2,10=2,28=1/1;
1/52=2,53=2032/20;
3/5=1,6=6,7=1,11=2,16=1,25=1,30=1,74=-5/1,2,3;
4//1;
5/5=2,38=5/2;
8/6=1,10=90,11=11/1;
9/8=-1,42=1/14;
6/7=2,8=2,9=2,10=2,28=1/1;
1/52=2,53=1022/20;
3/5=2,16=1,25=1,41=3900000/1,2,3;
4/35=1/1;
5/5=2,35=1,38=5/2;
8/6=1,10=90,11=11/1;
9/8=-1,42=1,49=10/14;
6/7=2,8=2,9=2,10=2,28=1/1;
1/52=2,53=3014/20;
99/5=1,9=1/99;"""

In [12]:
cntrl_test = read(azo_44_ts_xyz)
cntrl_test.set_calculator(Gaussian(label='cntrl_test', method='oniom(B3LYP/6-31G(d):pm6)', basis='oniom'))
cntrl_test_H = ylide_atoms + graph_atoms[3]
cntrl_test.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_test_H]]
cntrl_test.calc.oniom_coord_params['layer_mults'] = [1,1]
cntrl_test.calc.set_job(nodes=12, memory=12*1600, time=5, queue='pqmb', version='g09')
cntrl_test.calc.start()

/home/clyde/Project/Code/Python/gausspy/gaussian.py:1407: RuntimeWarning: Calculation already performed but incomplete, overwriting
  warnings.warn(RuntimeWarning('Calculation already performed but incomplete, overwriting'))


In [13]:
crt_test = read(azo_44_ts_xyz)
crt_test.set_calculator(Gaussian(label='crt_test', method='oniom(B3LYP/6-31G(d):pm6)', basis='oniom', non_std_route=trial_rt))
crt_test_H = ylide_atoms + graph_atoms[3]
crt_test.calc.oniom_coord_params['layers'] = [[no -1 for no in azo_ts_test_H]]
crt_test.calc.oniom_coord_params['layer_mults'] = [1,1]
crt_test.calc.set_job(nodes=12, memory=12*1600, time=5, queue='pqmb', version='g09')
crt_test.calc.start()

In [15]:
check_calcs([cntrl_test, crt_test])
mols_to_html([cntrl_test, crt_test])

cntrl_test,com,,Success
crt_test,com,,Success
